# This notebook represents the same functionality but run on Google colab

In [ ]:
!git clone https://github.com/bugsiesegal/harry-potter-remix-bot

In [ ]:
import sys
sys.path.append('/content/harry-potter-remix-bot/src')
from numpy import array
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.text import one_hot
from keras.models import load_model
import parameters
from data_preprocess import loaddata, remove_impuritys, one_hot_encode, one_hot_decode, number_encode, predict_text

In [ ]:
#STEP: Upload data to colab runtime environment
%mv '/content/1.txt' '/content/harry-potter-remix-bot/data/raw'

In [ ]:
#Pre-processing
books=loaddata(parameters.colab_datapath, 1)
text=remove_impuritys(books)
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [ ]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
## from keras.models import Sequential
from keras.layers import *

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=X_modified[0].shape,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Dense(64))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
x_train=X_modified[:int(len(X_modified)*0.8)]
y_train=Y_modified[:int(len(Y_modified)*0.8)]
x_test=X_modified[int(len(X_modified)*0.8):]
y_test=Y_modified[int(len(Y_modified)*0.8):]

In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=1024, validation_split=0.1)


In [ ]:
model.save('my_model.h5')
#Download to local machine
from google.colab import files
files.download("my_model.h5")

In [ ]:
predict_text(model, X[99])